### Продвинутые методы машинного обучения (1 семестр) 
### Индивидуальное задание по модулю «Визуализация данных»
#### **Выполнил:** Алексеев Арсалан Георгиевич

#### **Датасет №4:** Список персонажей DC Comics
https://www.kaggle.com/datasets/arunasivapragasam/dc-comics

#### Описание:
Мы запускаем свою линейку комиксов и нам нужно определить паттерн (набор черт, например: мужчина с голубыми глазами и с засекреченной личностью, но при этом злодей) пяти наиболее популярных (по числу появлений) персонажей.

#### Гипотеза:
Самые популярные персонажи часто похожи и можно из них выделить некие паттерны, на которые можно опираться при создании персонажей.

![DC comics image](https://assets.nationalnewswatch.com/wp-content/uploads/2013/12/superhero.jpg "DC comics image")

Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# python version 3.9.10

1. Проверим качество данных

In [2]:
# Считываем CSV file в pandas DataFrame
dc = pd.read_csv('data/dc-comics.csv')

# Создадим копию для дальнейшей работы
dc_initially = dc.copy()

In [3]:
# Ознакамливаемся с данными
dc.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,1237.0,"1940, April",1940.0


1.a Проверяем типы данных

In [4]:
# Смотрим общую информацию о данных
dc.info()

# Заметим, что только 3 первых столбца не имеют пропусков
# Тип данных столбца FIRST APPEARANCE - object, возможно нужно преобразование
# Остальные типы данных нормальные и в преобразовании не нуждаются

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6896 entries, 0 to 6895
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   page_id           6896 non-null   int64  
 1   name              6896 non-null   object 
 2   urlslug           6896 non-null   object 
 3   ID                4883 non-null   object 
 4   ALIGN             6295 non-null   object 
 5   EYE               3268 non-null   object 
 6   HAIR              4622 non-null   object 
 7   SEX               6771 non-null   object 
 8   ALIVE             6893 non-null   object 
 9   APPEARANCES       6541 non-null   float64
 10  FIRST APPEARANCE  6827 non-null   object 
 11  YEAR              6827 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 646.6+ KB


In [5]:
# Удалим неинформативные стобцы
dc.drop(columns=['page_id', 'urlslug', 'FIRST APPEARANCE'], inplace=True)

# Столбцы 'page_id', 'urlslug' не несут важную для исследования информацию
# Столбец 'FIRST APPEARANCE' дублирует год в столбце YEAR
# Оставим столбец YEAR, так как он уже находится в числовом формате

1.b Проверяем наличие пропусков

In [6]:
display(dc.isnull().tail())
cols_null_percent = dc.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

# Изучим более подробно столбцы с пропусками,
# попытаемся понять с чем связаны пропуски

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
6891,False,False,False,True,True,False,False,True,True
6892,False,False,False,True,True,False,False,True,True
6893,False,False,False,True,True,False,False,True,True
6894,False,False,False,True,True,False,False,True,True
6895,False,False,False,False,False,False,False,True,True


EYE            52.610209
HAIR           32.975638
ID             29.190835
ALIGN           8.715197
APPEARANCES     5.147912
SEX             1.812645
YEAR            1.000580
ALIVE           0.043503
dtype: float64

In [7]:
# Определим столбцы, наличие пропусков в которых может быть обосновано
dc.nunique()

# Список названий характеристик персонажа
dc_characteristics = ['ID', 'ALIGN', 'EYE', 'HAIR', 'SEX', 'ALIVE']
cols_null_for_cat = dc[dc_characteristics]

# Посмотрим для каждого столбца уникальные значения

for column in cols_null_for_cat:
    dc_cols_unique_values = dc[column].unique()
    print(f"Column: {column}\nUnique Values: {dc_cols_unique_values}\n")

# Наличие пропусков в этих столбцах вполне возможно и объяснимо
# особенностями самого персонажа, а не браком в данных, подробнее разберемся ниже

# Заметим также, что в столбце 'EYE' по ошибке попали значения из столбца 'HAIR'
# 'Auburn Hair'

Column: ID
Unique Values: ['Secret Identity' 'Public Identity' nan 'Identity Unknown']

Column: ALIGN
Unique Values: ['Good Characters' 'Bad Characters' 'Neutral Characters' nan
 'Reformed Criminals']

Column: EYE
Unique Values: ['Blue Eyes' 'Brown Eyes' 'Green Eyes' 'Purple Eyes' 'Black Eyes'
 'White Eyes' 'Red Eyes' 'Photocellular Eyes' 'Hazel Eyes' 'Amber Eyes'
 'Yellow Eyes' nan 'Grey Eyes' 'Pink Eyes' 'Violet Eyes' 'Gold Eyes'
 'Orange Eyes' 'Auburn Hair']

Column: HAIR
Unique Values: ['Black Hair' 'Brown Hair' 'White Hair' 'Blond Hair' 'Red Hair' nan
 'Green Hair' 'Strawberry Blond Hair' 'Grey Hair' 'Silver Hair'
 'Orange Hair' 'Purple Hair' 'Gold Hair' 'Blue Hair' 'Reddish Brown Hair'
 'Pink Hair' 'Violet Hair' 'Platinum Blond Hair']

Column: SEX
Unique Values: ['Male Characters' 'Female Characters' nan 'Genderless Characters'
 'Transgender Characters']

Column: ALIVE
Unique Values: ['Living Characters' 'Deceased Characters' nan]



In [8]:
# Посмотрим на строки, где произошла ошибка со значениями
dc.loc[dc['EYE'] == 'Auburn Hair']

# Действительно произошла ошибка, в столбце 'HAIR' данные отсутствуют

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
872,Marcia King (New Earth),Public Identity,Good Characters,Auburn Hair,NaN,Female Characters,Living Characters,32.0,1984.0
1776,"Anthony Angelo, Jr. (New Earth)",Public Identity,Neutral Characters,Auburn Hair,NaN,Male Characters,Living Characters,14.0,1989.0
3600,Tawna (New Earth),NaN,Good Characters,Auburn Hair,NaN,Female Characters,Living Characters,5.0,1998.0
3623,Razerkut (New Earth),Secret Identity,Bad Characters,Auburn Hair,NaN,Female Characters,Living Characters,5.0,1995.0
3627,Carla Draper (New Earth),Public Identity,Bad Characters,Auburn Hair,NaN,Female Characters,Living Characters,5.0,1994.0
4070,Madolyn Corbett (New Earth),NaN,Neutral Characters,Auburn Hair,NaN,Female Characters,Deceased Characters,4.0,1995.0
5346,Sally Milton (New Earth),Public Identity,NaN,Auburn Hair,NaN,Female Characters,Living Characters,2.0,1988.0


In [9]:
# Временно запомним номера ошибочных строк
dc_index_tp = dc[dc['EYE'] == 'Auburn Hair'].index

# Заменим ошибочные ячейки на пропуски
dc.loc[dc_index_tp, ['EYE']] = np.nan
dc.loc[dc_index_tp, ['HAIR']] = 'Auburn Hair'

# Проверим результат замены
dc.loc[dc_index_tp]

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
872,Marcia King (New Earth),Public Identity,Good Characters,NaN,Auburn Hair,Female Characters,Living Characters,32.0,1984.0
1776,"Anthony Angelo, Jr. (New Earth)",Public Identity,Neutral Characters,NaN,Auburn Hair,Male Characters,Living Characters,14.0,1989.0
3600,Tawna (New Earth),NaN,Good Characters,NaN,Auburn Hair,Female Characters,Living Characters,5.0,1998.0
3623,Razerkut (New Earth),Secret Identity,Bad Characters,NaN,Auburn Hair,Female Characters,Living Characters,5.0,1995.0
3627,Carla Draper (New Earth),Public Identity,Bad Characters,NaN,Auburn Hair,Female Characters,Living Characters,5.0,1994.0
4070,Madolyn Corbett (New Earth),NaN,Neutral Characters,NaN,Auburn Hair,Female Characters,Deceased Characters,4.0,1995.0
5346,Sally Milton (New Earth),Public Identity,NaN,NaN,Auburn Hair,Female Characters,Living Characters,2.0,1988.0


Как мы увидим ниже, у персонажа Susan Dearbon (New Earth) в строке EYE: NaN, но согласно Wiki, данный персонаж и правда ослеп, отбрасывать подобные пропуски мы не будем. Но для сравнения Samuel Morgan (New Earth) имеет пропуск в столбце EYE,
обусловленный старым стилем рисовки (глаза обазначены просто точкой).

In [10]:
cols_null_percent = dc.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)

# посмотрим начиная с какого персонажа возникают пропуски
display(dc.loc[dc['EYE'].isnull()].head())

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
95,Susan Dearbon (New Earth),Public Identity,Good Characters,NaN,Black Hair,Female Characters,Deceased Characters,235.0,1961.0
118,James Ewell Brown Stuart (New Earth),Public Identity,Good Characters,NaN,Brown Hair,Male Characters,Deceased Characters,212.0,1961.0
122,Wolfgang Winks (New Earth),Public Identity,Good Characters,NaN,Black Hair,Male Characters,Living Characters,206.0,1942.0
164,Andre Chavard (New Earth),Public Identity,Good Characters,NaN,Brown Hair,Male Characters,Living Characters,164.0,1942.0
181,Samuel Morgan (New Earth),NaN,Good Characters,NaN,Red Hair,Male Characters,Deceased Characters,155.0,1937.0


Так как столбец EYE имеет больше всего пропусков (более 50%) на его примере разберемся, с чем связаны пропуски и насколько критично удаление строк с пропусками.

In [11]:
# общее количество появлений персонажей с отсутствующими данными в столбце 'EYE'
dc_eye_nan = dc.loc[dc['EYE'].isnull()]['APPEARANCES'].sum()

# процент появления персонажей с отсутствующими данными в столбце 'EYE' от общего суммы появлений 
dc_eye_nan_perc = 100 * dc_eye_nan/dc['APPEARANCES'].sum()

print(f'процент появления персонажей с отсутствующими данными в столбце EYE: {round(dc_eye_nan_perc,2)}')

процент появления персонажей с отсутствующими данными в столбце EYE: 17.37


Рассмотрим другие столбцы с пропусками

In [12]:
display(dc.loc[dc['ALIGN'].isnull()].head())

# пропуски в данном столбце скорее всего несут ошибку, 
# ввиду того определить добрый или злой персонаж не является затруднительным
# согласно wiki для большинства персонажей данный параметр определен

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
22,Ted Grant (New Earth),Secret Identity,NaN,Blue Eyes,Black Hair,Male Characters,Living Characters,605.0,1942.0
27,Ralph Dibny (New Earth),Public Identity,NaN,Blue Eyes,Red Hair,Male Characters,Deceased Characters,558.0,1960.0
33,Rex Mason (New Earth),Secret Identity,NaN,Black Eyes,NaN,Male Characters,Living Characters,470.0,1965.0
35,Aztar (New Earth),Secret Identity,NaN,White Eyes,NaN,Male Characters,Living Characters,436.0,1940.0
36,Theodore Kord (New Earth),Secret Identity,NaN,Blue Eyes,Brown Hair,Male Characters,Deceased Characters,429.0,1986.0


In [13]:
display(dc.loc[dc['HAIR'].isnull()].head())

# пропуски в данной графе скорее всего связаны с тем, что персонаж либо лысый, 
# либо не снимает головной убор/костюм, либо не имеет волос изначально

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
18,Raymond Palmer (New Earth),Public Identity,Good Characters,Brown Eyes,NaN,Male Characters,Living Characters,706.0,1961.0
19,Alexander Luthor (New Earth),Public Identity,Bad Characters,Green Eyes,NaN,Male Characters,Living Characters,677.0,1986.0
31,Franklin Rock (New Earth),Public Identity,Good Characters,Blue Eyes,NaN,Male Characters,Living Characters,492.0,1959.0
33,Rex Mason (New Earth),Secret Identity,NaN,Black Eyes,NaN,Male Characters,Living Characters,470.0,1965.0
35,Aztar (New Earth),Secret Identity,NaN,White Eyes,NaN,Male Characters,Living Characters,436.0,1940.0


In [14]:
display(dc.loc[dc['SEX'].isnull()].head())

# насколько мы видим, персонажи не имеющие параматеров EYE, HAIR, SEX представляют из себя не человека, 
# а какую-то другую форму жизни, поэтому это тоже не является ошибкой,

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
242,Khaji Da (New Earth),Secret Identity,Good Characters,NaN,NaN,NaN,Living Characters,121.0,1964.0
343,Rosabelle Mendez (New Earth),Secret Identity,Good Characters,Red Eyes,NaN,NaN,Deceased Characters,86.0,1991.0
352,Parallax (New Earth),Secret Identity,Bad Characters,NaN,NaN,NaN,Living Characters,84.0,1994.0
514,Chemo (New Earth),Public Identity,Bad Characters,NaN,NaN,NaN,Living Characters,58.0,1962.0
628,Ion (New Earth),Secret Identity,Good Characters,Green Eyes,NaN,NaN,Deceased Characters,45.0,2006.0


Так как даже на примере со столбцом EYE, который имеет больше всего пропусков, но процент появления персонажей с пропусками низкий (около 17%), то исключить строки с пропусками без потери информативности данных, однако не все.


Мы отбрасываем строки с числом пропусков от 2 и более, так как данные строки заполнены неверно или неинформативно.
Строки, содержащие 1 пропуск скорее всего имеют его, в виду особенностей персонажа, на паре примеров мы это проверили.

In [15]:
m = dc.shape[1] #число столбцов
dc = dc.dropna(axis=0,thresh=m-1)

#выводим долю пропусков после преобразования
print(f'Результирующее число строк ДО: {dc_initially.shape[0]}')
print(f'Результирующее число строк ПОСЛЕ: {dc.shape[0]}')
display(dc.isnull().mean())


Результирующее число строк ДО: 6896
Результирующее число строк ПОСЛЕ: 4070


name           0.000000
ID             0.081327
ALIGN          0.034644
EYE            0.260934
HAIR           0.087715
SEX            0.000246
ALIVE          0.000000
APPEARANCES    0.017445
YEAR           0.002457
dtype: float64

In [16]:
# Удалим строки содержащие пропуски в столбцах 'APPEARANCES', 'ID' и 'YEAR',
# Наличие информации в этих столбцах необходимо, а в столбце ID есть отдельная позиция 'Identity Unknown'
dc.dropna(subset=['APPEARANCES', 'YEAR','ID'], inplace=True)

# В столбце ID есть строки Identity Unknown, заменим их на Unknown
dc['ID'] = dc['ID'].replace('Identity Unknown', 'Unknown')

#выводим долю пропусков после преобразования
print(f'Результирующее число строк ДО: {dc_initially.shape[0]}')
print(f'Результирующее число строк ПОСЛЕ: {dc.shape[0]}')
display(dc.isnull().mean())


Результирующее число строк ДО: 6896
Результирующее число строк ПОСЛЕ: 3658


name           0.000000
ID             0.000000
ALIGN          0.038546
EYE            0.290323
HAIR           0.097594
SEX            0.000273
ALIVE          0.000000
APPEARANCES    0.000000
YEAR           0.000000
dtype: float64

In [17]:
# Заменим оставшиеся пропуски на 'Unknown' для удобства дальнейшей работы
for column in dc:
    dc[column].fillna('Unknown', inplace=True)

1.c Проверяем наличие дубликатов

In [18]:
dupl_columns = list(dc.columns)
mask = dc.duplicated(subset=dupl_columns)
dc_duplicates = dc[mask]
print(f'Число найденных дубликатов: {dc_duplicates.shape[0]}')
dc = dc.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {dc.shape[0]}')

# Дубликатов не обнаружено

Число найденных дубликатов: 0
Результирующее число записей: 3658


2.Рассчитаем основные показатели

In [19]:
# Определим 5 самых популярных персонажей
dc_hero_top5 = dc.sort_values('APPEARANCES', ascending=False).head(10)

# Сохраним характеристики 5 самых популярных персонажей 
dc_character_place1 = (list(dc_hero_top5.loc[0:4,dc_characteristics]))
dc_character_place2 = (list(dc_hero_top5.loc[1,dc_characteristics]))
dc_character_place3 = (list(dc_hero_top5.loc[2,dc_characteristics]))
dc_character_place4 = (list(dc_hero_top5.loc[3,dc_characteristics]))
dc_character_place5 = (list(dc_hero_top5.loc[4,dc_characteristics]))

print(dc_hero_top5.loc[0:19])


                             name               ID            ALIGN  \
0            Batman (Bruce Wayne)  Secret Identity  Good Characters   
1           Superman (Clark Kent)  Secret Identity  Good Characters   
2      Green Lantern (Hal Jordan)  Secret Identity  Good Characters   
3        James Gordon (New Earth)  Public Identity  Good Characters   
4     Richard Grayson (New Earth)  Secret Identity  Good Characters   
5     Wonder Woman (Diana Prince)  Public Identity  Good Characters   
6          Aquaman (Arthur Curry)  Public Identity  Good Characters   
7       Timothy Drake (New Earth)  Secret Identity  Good Characters   
8  Dinah Laurel Lance (New Earth)  Public Identity  Good Characters   
9             Flash (Barry Allen)  Secret Identity  Good Characters   

          EYE        HAIR                SEX              ALIVE  APPEARANCES  \
0   Blue Eyes  Black Hair    Male Characters  Living Characters       3093.0   
1   Blue Eyes  Black Hair    Male Characters  Living Chara

6.Перевод и вывод результата

In [20]:
# для автоматического перевода на русский язык, воспользуемся Googletrans 

import googletrans
from googletrans import Translator

In [21]:
# генерируем список словарей для будущей замены
translator = Translator()
lists_translate = []

for _ in range(len(dc_characteristics)):
        # для каждой характеристики создадим свой словарь для перевода

    for characteristic in dc_characteristics:
        # в качестве ключей используем значения характеристик
        dc_dict_keys = list(dc[characteristic].unique())
        dc_dict_values = list()
        for k in range(len(dc_dict_keys)):
          dc_dict_values.append(translator.translate(str(dc_dict_keys[k]), dest='ru').text)
        lists_translate.append({dc_dict_keys[i]: dc_dict_values[i] for i in range(len(dc_dict_keys))})

In [22]:
print(lists_translate[0])

{'Secret Identity': 'Секретная личность', 'Public Identity': 'Общественная идентичность', 'Unknown': 'Неизвестный'}


In [23]:
#lists = [{} for _ in range(len(dc_characteristics))]

#lists_translate = []
#for i in range(len(dc_characteristics)):
    #lists_translate.append({})
    #lists_translate[i] = [i]
    #print(lists_translate)
    #f"dict_{i+1}" = lists_translate[i]



In [24]:
# Преобразуем списки характеристик в более понятные предложения на русском языке

# Создадим словари для замены слов
dict_ID = {
    'Secret Identity': 'с засекреченной личностью',
    'Public Identity': 'с публичной личностью',
    'Unknown': '',
}

dict_ALIGN = {
    'Good Characters': 'добрый персонаж',
    'Bad Characters': 'злой персонаж',
    'Neutral Characters': 'нейтральный персонаж',
    'Reformed Criminals': 'персонаж встал на путь исправления',
    'Unknown': '',
}

dict_EYE = {
    'Blue Eyes': 'Голубые глаза',
    'Brown Eyes': 'Карие глаза',
    'Green Eyes': 'Зеленые глаза',
    'Purple Eyes': 'Пурпурные глаза',
    'Black Eyes': 'Черные глаза',
    'White Eyes': 'Белые глаза',
    'Red Eyes': 'Красные глаза',
    'Photocellular Eyes': 'Фотоэлементные глаза',
    'Hazel Eyes': 'Ореховые глаза',
    'Amber Eyes': 'Янтарные глаза',
    'Yellow Eyes': 'Желтые глаза',
    'Grey Eyes': 'Серые глаза',
    'Pink Eyes': 'Розовые глаза',
    'Violet Eyes': 'Фиалковые глаза',
    'Gold Eyes': 'Золотистые глаза',
    'Orange Eyes': 'Оранжевые глаза',
    'Unknown': '', ''
}

dict_ALIGN = {
    'Good Characters': 'добрый персонаж',
    'Bad Characters': 'злой персонаж',
    'Neutral Characters': 'нейтральный персонаж',
    'Reformed Criminals': 'персонаж встал на путь исправления',
    'Unknown': '',
}
# Создадим функцию для замены слов

def eng_to_rus(word):

Column: EYE
Unique Values: ['Blue Eyes' 'Brown Eyes' 'Green Eyes' 'Purple Eyes' 'Black Eyes'
 'White Eyes' 'Red Eyes' 'Photocellular Eyes' 'Hazel Eyes' 'Amber Eyes'
 'Yellow Eyes' nan 'Grey Eyes' 'Pink Eyes' 'Violet Eyes' 'Gold Eyes'
 'Orange Eyes' 'Auburn Hair']

Column: HAIR
Unique Values: ['Black Hair' 'Brown Hair' 'White Hair' 'Blond Hair' 'Red Hair' nan
 'Green Hair' 'Strawberry Blond Hair' 'Grey Hair' 'Silver Hair'
 'Orange Hair' 'Purple Hair' 'Gold Hair' 'Blue Hair' 'Reddish Brown Hair'
 'Pink Hair' 'Violet Hair' 'Platinum Blond Hair']

Column: SEX
Unique Values: ['Male Characters' 'Female Characters' nan 'Genderless Characters'
 'Transgender Characters']

Column: ALIVE
Unique Values: ['Living Characters' 'Deceased Characters' nan]


SyntaxError: invalid syntax (964805114.py, line 36)

In [ ]:
dc_HAIR = dc.groupby('HAIR')['APPEARANCES'].max().sort_values(ascending=False)
dc_HAIR.head()

HAIR
Black Hair    3093.0
Brown Hair    1565.0
White Hair    1316.0
Blond Hair    1121.0
Red Hair       951.0
Name: APPEARANCES, dtype: float64

In [ ]:
dc.groupby(['ID', 'ALIGN', 'EYE', 'HAIR', 'SEX', 'ALIVE'])['APPEARANCES'].max().unstack().head()

ALIVE                                                                    Deceased Characters  \
ID              ALIGN          EYE         HAIR       SEX                                      
Public Identity Bad Characters Auburn Hair Unknown    Female Characters                  NaN   
                               Black Eyes  Black Hair Female Characters                 25.0   
                                                      Male Characters                  142.0   
                                                      Unknown                            1.0   
                                           Blond Hair Male Characters                    6.0   

ALIVE                                                                    Living Characters  \
ID              ALIGN          EYE         HAIR       SEX                                    
Public Identity Bad Characters Auburn Hair Unknown    Female Characters                5.0   
                               Black Eyes  Black Hair Female Characters               61.0   
                                                      Male Characters                158.0   
                                                      Unknown                          NaN   
                                           Blond Hair Male Characters                  3.0   

ALIVE                                                                    Unknown  
ID              ALIGN          EYE         HAIR       SEX                         
Public Identity Bad Characters Auburn Hair Unknown    Female Characters      NaN  
                               Black Eyes  Black Hair Female Characters      NaN  
                                                      Male Characters        NaN  
                                                      Unknown                NaN  
                                           Blond Hair Male Characters        NaN

In [ ]:
dc.pivot_table(
    values='APPEARANCES',
    index=['ID', 'ALIGN', 'EYE', 'HAIR', 'SEX', 'ALIVE'],
    columns='Regionname',
    aggfunc='max',
    fill_value=0
)

In [ ]:
display(dc)

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
0,Batman (Bruce Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3093.0,1939.0
1,Superman (Clark Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2496.0,1986.0
2,Green Lantern (Hal Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,Living Characters,1565.0,1959.0
3,James Gordon (New Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,Living Characters,1316.0,1987.0
4,Richard Grayson (New Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,1237.0,1940.0
...,...,...,...,...,...,...,...,...,...
6527,Gypper Zilich (New Earth),Unknown,Unknown,Unknown,Black Hair,Male Characters,Living Characters,1.0,1941.0
6528,Robert Mason (New Earth),Secret Identity,Neutral Characters,Black Eyes,Black Hair,Male Characters,Living Characters,1.0,1941.0
6529,Diablo the Horse (New Earth),Unknown,Good Characters,Unknown,Black Hair,Male Characters,Living Characters,1.0,1940.0
6530,Scott Scanlon (New Earth),Public Identity,Unknown,Unknown,Blond Hair,Male Characters,Living Characters,1.0,1938.0
